# Entender los tipos de datos en Python

La ciencia y la computación eficaces basadas en datos requieren comprender cómo se almacenan y manipulan los datos.
Esta sección describe y contrasta cómo se manejan las arrays de datos en el propio lenguaje Python, y cómo NumPy mejora esto.
Entender esta diferencia es fundamental para comprender gran parte del material del resto del libro.

Los usuarios de Python a menudo se sienten atraídos por su facilidad de uso, una de cuyas piezas es la tipificación dinámica.
Mientras que un lenguaje de tipado estático como C o Java requiere que cada variable sea declarada explícitamente, un lenguaje de tipado dinámico como Python se salta esta especificación. Por ejemplo, en C podrías especificar una operación particular de la siguiente manera:


```C
/* Código C */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

Mientras que en Python la operación equivalente podría escribirse así:

```python
# Código Python
result = 0
for i in range(100):
    result += i
```

Fíjate en la principal diferencia: en C, los tipos de datos de cada variable se declaran explícitamente, mientras que en Python los tipos se infieren dinámicamente. Esto significa, por ejemplo, que podemos asignar cualquier tipo de datos a cualquier variable:

```python
# Código Python
x = 4
x = "four"
```

Aquí hemos cambiado el contenido de ``x`` de un integer a una string. Lo mismo en C llevaría (dependiendo de la configuración del compilador) a un error de compilación u otras consecuencias no deseadas:

```C
/* Código C */
int x = 4;
x = "cuatro";  // FAILS
```

Este tipo de flexibilidad es una pieza que hace que Python y otros lenguajes de tipado dinámico sean convenientes y fáciles de usar.
Entender *cómo* funciona esto es una pieza importante para aprender a analizar datos de forma eficiente y eficaz con Python.
Pero lo que esta flexibilidad de tipos también señala es el hecho de que las variables de Python son más que su valor; también contienen información extra sobre el tipo del valor. Exploraremos más esto en las secciones siguientes.

## Un integer en Python es más que un entero

La implementación estándar de Python está escrita en C.
Esto significa que cada objeto de Python es simplemente una estructura de C inteligentemente disfrazada, que contiene no sólo su valor, sino también otra información. Por ejemplo, cuando definimos un integer en Python, como ``x = 10000``, ``x`` no es sólo un entero "en bruto". En realidad es un puntero a una estructura compuesta en C, que contiene varios valores.
Mirando el código fuente de Python 3.4, encontramos que la definición del tipo integer (largo) se ve efectivamente así (una vez que las macros C se expanden):



```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Un único integer en Python 3.4 contiene en realidad cuatro piezas:

- ``ob_refcnt``, un recuento de referencias que ayuda a Python a manejar silenciosamente la asignación y desasignación de memoria
- ``ob_type``, que codifica el tipo de la variable
- ``ob_size``, que especifica el tamaño de los siguientes miembros de datos
- ``ob_digit``, que contiene el valor entero real que esperamos que represente la variable de Python.

Esto significa que hay una cierta sobrecarga en el almacenamiento de un integer en Python en comparación con un integer en un lenguaje compilado como C, como se ilustra en la siguiente figura:

![Integer Memory Layout](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/cint_vs_pyint.png?raw=1)

Aquí ``PyObject_HEAD`` es la parte de la estructura que contiene el recuento de referencias, el código de tipo y otras piezas mencionadas anteriormente.

Fíjate en la diferencia: un integer en C es esencialmente una etiqueta para una posición en memoria cuyos bytes codifican un valor integer.
Un integer de Python es un puntero a una posición en memoria que contiene toda la información del objeto de Python, incluyendo los bytes que contienen el valor del integer.
Esta información extra en la estructura de los integer de Python es lo que permite codificar Python de forma tan libre y dinámica.
Sin embargo, toda esta información adicional en los tipos de Python tiene un coste, que se hace especialmente evidente en las estructuras que combinan muchos de estos objetos.

## Un list de Python es más que una lista

Consideremos ahora lo que sucede cuando usamos una estructura de datos de Python que contiene muchos objetos de Python.
El contenedor multi-elemento mutable estándar en Python el list.
Podemos crear un list de integers de la siguiente manera:

In [ ]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
type(L[0])

int

O, de forma similar, un list de strings:

In [ ]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
type(L2[0])

str

Gracias a la tipificación dinámica de Python, podemos incluso crear lists heterogéneas:

In [ ]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

Pero esta flexibilidad tiene un coste: para permitir estos tipos flexibles, cada elemento del list debe contener su propia información de tipo, número de referencias y otra información, es decir, cada elemento es un objeto Python completo.
En el caso especial de que todas las variables sean del mismo tipo, mucha de esta información es redundante: puede ser mucho más eficiente almacenar los datos en un array de tipo fijo.
La diferencia entre un list de tipo dinámico y un array de tipo fijo (estilo NumPy) se ilustra en la siguiente figura:

![Array Memory Layout](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/array_vs_list.png?raw=1)

A nivel de implementación, el array contiene esencialmente un único puntero a un bloque contiguo de datos.
El list de Python, por otro lado, contiene un puntero a un bloque de punteros, cada uno de los cuales apunta a su vez a un objeto completo de Python como el integer de Python que vimos antes.
De nuevo, la ventaja del list es la flexibilidad: como cada elemento del list es una estructura completa que contiene tanto datos como información de tipo, el list puede llenarse con datos de cualquier tipo deseado.
Los arrays de tipo fijo al estilo de NumPy carecen de esta flexibilidad, pero son mucho más eficientes para almacenar y manipular datos.

## Arrays de tipo fijo en Python

Python ofrece varias opciones para almacenar datos en buffers de datos eficientes y de tipo fijo.
El módulo incorporado ``array`` (disponible desde Python 3.3) se puede utilizar para crear arrays densos de un tipo uniforme:

In [ ]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Aquí ``'i'`` es un código de tipo que indica que los contenidos son integers.

Mucho más útil, sin embargo, es el objeto ``ndarray`` del paquete NumPy.
Mientras que el objeto ``array`` de Python proporciona un almacenamiento eficiente de datos basados en arrays, NumPy añade a esto *operaciones* eficientes sobre esos datos.
Exploraremos estas operaciones en secciones posteriores; aquí demostraremos varias formas de crear un array de NumPy.

Empezaremos con la importación estándar de NumPy, bajo el alias ``np``:

In [ ]:
import numpy as np

## Creación de arrays a partir de lists de Python

En primer lugar, podemos utilizar ``np.array`` para crear arrays a partir de lists de Python:

In [ ]:
# integer array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Recuerde que, a diferencia de los lists de Python, NumPy está limitado a los arrays que contienen el mismo tipo.
Si los tipos no coinciden, NumPy hará un upcast si es posible (aquí, los integers son upcast a punto flotante):

In [ ]:
np.array([3.14, 4, 2, 3])

array([ 3.14,  4.  ,  2.  ,  3.  ])

Si queremos establecer explícitamente el tipo de datos del array resultante, podemos utilizar la palabra clave ``dtype``:

In [ ]:
np.array([1, 2, 3, 4], dtype='float32')

array([ 1.,  2.,  3.,  4.], dtype=float32)

Finalmente, a diferencia de los lists de Python, los arrays de NumPy pueden ser explícitamente multidimensionales; aquí hay una forma de inicializar un array multidimensional usando un list de lists:

In [ ]:
# los lists anidados dan lugar a arrays multidimensionales
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

Los lists interiores se tratan como filas del array bidimensional resultante.

## Creación de Arrays desde cero

Especialmente para los arrays más grandes, es más eficiente crear arrays desde cero utilizando las rutinas incorporadas en NumPy.
Aquí hay varios ejemplos:

In [ ]:
# Crea un array de integers de longitud 10 llena de ceros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# Crea un array de 3x5 puntos flotantes llena de unos
np.ones((3, 5), dtype=float)

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [ ]:
# Crea un array de 3x5 llena de 3.14
np.full((3, 5), 3.14)

array([[ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14]])

In [ ]:
# Crea un array lleno de una secuencia lineal
# Empezando en 0, terminando en 20, pasando por 2
# (esto es similar a la función incorporada range())
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [ ]:
# Crear un array de cinco valores espaciados uniformemente entre 0 y 1
np.linspace(0, 1, 5)

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [ ]:
# Crear un array de 3x3 de valores distribuidos uniformemente
# valores aleatorios entre 0 y 1
np.random.random((3, 3))

array([[ 0.99844933,  0.52183819,  0.22421193],
       [ 0.08007488,  0.45429293,  0.20941444],
       [ 0.14360941,  0.96910973,  0.946117  ]])

In [ ]:
# Crear un array de 3x3 de valores aleatorios normalmente distribuidos
# con media 0 y desviación estándar 1
np.random.normal(0, 1, (3, 3))

array([[ 1.51772646,  0.39614948, -0.10634696],
       [ 0.25671348,  0.00732722,  0.37783601],
       [ 0.68446945,  0.15926039, -0.70744073]])

In [ ]:
# Crea un array 3x3 de integers aleatorios en el intervalo [0, 10)
np.random.randint(0, 10, (3, 3))

array([[2, 3, 4],
       [5, 7, 8],
       [0, 5, 0]])

In [ ]:
# Crear un array identidad 3x3
np.eye(3)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [ ]:
# Crea un array no inicializado de tres integers
# Los valores serán los que ya existan en esa posición de memoria
np.empty(3)

array([ 1.,  1.,  1.])

## Tipos de datos estándar de NumPy

Los arrays de NumPy contienen valores de un solo tipo, por lo que es importante tener un conocimiento detallado de esos tipos y sus limitaciones.
Como NumPy está construido en C, los tipos serán familiares para los usuarios de C, Fortran y otros lenguajes relacionados.

Los tipos de datos estándar de NumPy se enumeran en la siguiente tabla.
Tenga en cuenta que cuando se construye un array, se pueden especificar utilizando un string:

```python
np.zeros(10, dtype='int16')
```

O utilizando el objeto NumPy asociado:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Booleano (Verdadero o Falso) almacenado como un byte. |
| ``int_``      | Tipo Integer por defecto (igual que C ``long``; normalmente ``int64`` o ``int32````int64`` or ``int32``)| 
| ``intc``      | Idéntico a C ``int`` (normalmente ``int32`` o ``int64``)| 
| ``intp``      | Número Integer utilizado para la indexación (igual que el ``ssize_t`` de C; normalmente es ``int32`` o ``int64``)| 
| ``int8``      | Byte (-128 a 127)| 
| ``int16``     | Integer (-32768 a 32767)|
| ``int32``     | Integer (-2147483648 a 2147483647)|
| ``int64``     | Integer (-9223372036854775808 a 9223372036854775807)| 
| ``uint8``     | Integer sin signo (0 a 255)| 
| ``uint16``    | Integer sin signo (0 a 65535)| 
| ``uint32``    | Integer sin signo (0 a 4294967295)| 
| ``uint64``    | Integer sin signo (0 a 18446744073709551615)| 
| ``float_``    | Abreviatura de ``float64``.| 
| ``float16``   | Float de media precisión: bit de signo, 5 bits de exponente, 10 bits de mantissa| 
| ``float32``   | Float de precisión única: bit de signo, exponente de 8 bits, mantissa de 23 bits| 
| ``float64``   | Float de doble precisión: bit de signo, exponente de 11 bits, mantissa de 52 bits | 
| ``complex_``  | Abreviatura de ``complex128``.| 
| ``complex64`` | Número complejo, representado por dos floats de 32 bits | 
| ``complex128``| Número complejo, representado por dos floats de 64 bits | 

Es posible especificar tipos más avanzados, como por ejemplo, especificar números big o little endian; para más información, consulte la sección [Documentación NumPy](http://numpy.org/).
NumPy también admite tipos de datos compuestos, que se tratarán en [Datos estructurados: Arrays estructurados de NumPy](02.09-Datos-Estructurados-NumPy.ipynb).